# Summarize DSSP files 

This script will read a series of dssp output files and generate a summary for each that I could try to relate to the HET / HM ratio in the simulations. For reference:

- H = alpha-helix
- B = beta-bridge residue
- E = extended strand (in beta ladder)
- G = 3/10-helix
- I = 5-helix
- T = H-bonded turn
- S = bend


In [1]:
# Load libraries
import csv
import numpy as np
import pandas as pd
import re
import glob
import os

In [2]:
code_dict = {
    'H':'Alpha helix',
    'B':'Beta bridge',
    'E':'Beta ladder', 
    'G':'3/10 helix',
    'I':'5-helix',
    'T':'H-bonded turn',
    'S':'Bend'
}

In [3]:
## Define some functions
def parse_pdb_line(pdb_line):
    '''This function will receive a line from a PDB file and parse it as a list. It will do so based on the
    PDB format explanation from this site:

    https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html.
    '''
    atom = pdb_line[0:4].strip(' ')
    atom_num = pdb_line[6:11].strip(' ')
    atom_name = pdb_line[12:16].strip(' ')
    resname = pdb_line[17:20].strip(' ')
    chain = pdb_line[21]
    res_num = pdb_line[22:27].strip(' ')
    x = pdb_line[30:38].strip(' ')
    y = pdb_line[38:46].strip(' ')
    z = pdb_line[46:54].strip(' ')
    bfactor = pdb_line[60:66].strip(' ')

    return [atom, atom_num, atom_name, resname, chain, res_num, x, y, z, bfactor]





## Need to adapt this block and make it loop through the list of structures

In [17]:
## Folder with the DSSP results
folder_path = '../../Data/DSSP'

# Get the list of files in that folder
file_list = glob.glob(os.path.join(folder_path, '*dssp'))


104
['../../Data/DSSP/4o48_bio_check.dssp', '../../Data/DSSP/1b57_bio_check.dssp', '../../Data/DSSP/1qq2_bio_check.dssp', '../../Data/DSSP/1dth_bio_check.dssp', '../../Data/DSSP/1m7p_bio_check.dssp', '../../Data/DSSP/1ukw_bio_check.dssp', '../../Data/DSSP/2raj_bio_check.dssp', '../../Data/DSSP/2p53_bio_check.dssp', '../../Data/DSSP/5rfd_bio_check.dssp', '../../Data/DSSP/1ifu_bio_check.dssp', '../../Data/DSSP/3wx4_bio_check.dssp', '../../Data/DSSP/1hg2_bio_check.dssp', '../../Data/DSSP/3ib7_bio_check.dssp', '../../Data/DSSP/1nmz_bio_check.dssp', '../../Data/DSSP/2rl2_bio_check.dssp', '../../Data/DSSP/1a72_bio_check.dssp', '../../Data/DSSP/1bkz_bio_check.dssp', '../../Data/DSSP/4cvt_bio_check.dssp', '../../Data/DSSP/2xz2_bio_check.dssp', '../../Data/DSSP/1cpj_bio_check.dssp', '../../Data/DSSP/3tgs_bio_check.dssp', '../../Data/DSSP/6cso_bio_check.dssp', '../../Data/DSSP/3ulh_bio_check.dssp', '../../Data/DSSP/1pzw_bio_check.dssp', '../../Data/DSSP/3gi4_bio_check.dssp', '../../Data/DSSP/1a4

In [21]:
df_used_structures = pd.read_csv('../../Data/final_104_structures.txt', sep = '\t', names = ['PDB_ID'])
used_structures = [entry for entry in df_used_structures['PDB_ID']]
used_structures

['1gpr',
 '1ifu',
 '1gv3',
 '1a8l',
 '1ai2',
 '4cvt',
 '2raj',
 '3r2v',
 '1ok9',
 '1d0q',
 '1juv',
 '3wx4',
 '1a4b',
 '4rfp',
 '5abr',
 '19hc',
 '2o4c',
 '1m7p',
 '1dth',
 '1uuf',
 '3ib7',
 '1cpj',
 '4wpe',
 '1c1f',
 '1bbh',
 '5c94',
 '2i4z',
 '4p16',
 '4fgw',
 '3h65',
 '1pzw',
 '4zvl',
 '1g99',
 '1bcg',
 '1m6d',
 '3gi4',
 '1edt',
 '3euo',
 '1am2',
 '4o48',
 '1nmz',
 '1hg2',
 '1b57',
 '4z04',
 '2vtk',
 '5hbi',
 '1h0c',
 '4z5z',
 '3u2g',
 '2pih',
 '4yzg',
 '1a72',
 '3naq',
 '1ukw',
 '2rl2',
 '6ftq',
 '5bj4',
 '2xz2',
 '5rfd',
 '3cxk',
 '3zof',
 '1cei',
 '1f05',
 '1alu',
 '4cwd',
 '1oi2',
 '1mk4',
 '3bac',
 '1ge7',
 '6cso',
 '1mi8',
 '3w5z',
 '4cmd',
 '1nxp',
 '1hpc',
 '2p09',
 '1c8t',
 '1qq2',
 '4fum',
 '3tgs',
 '1b7e',
 '1m38',
 '7d83',
 '1asb',
 '1asu',
 '1i49',
 '1at0',
 '2b18',
 '1btm',
 '4nak',
 '1eye',
 '2hqv',
 '1bdo',
 '2p53',
 '1bkz',
 '2imt',
 '1p6o',
 '3ot2',
 '3ulh',
 '3f4d',
 '1ang',
 '2dfj',
 '5hke',
 '3rzn']

In [22]:
out_folder = os.path.join(folder_path, 'Clean_tables')

os.makedirs(out_folder, exist_ok = True)

path_interfaces = '../../Data/Structures/004_interfaces/'

summary_dict = {}
summary_dict_interfaces = {}

# Loop through the files
for infile in file_list:
    # Extract the pdb id from the file name
    file_basename = os.path.basename(infile)
    pdb_id = str(file_basename.split('_')[0])
    # print(pdb_id)
    
    # Skip if this structure is not in the final data set
    if not pdb_id in used_structures:
        continue
    
    handle = open(infile, 'r')

    # Start a dictionary to keep track of the values
    sec_struc_annotations = {}

    # A boolean to help us know when to start looking at the data
    bool_data = False
    
    # Loop through the lines
    for line in handle:

        # Looking for the start of the data
        if not bool_data:
            if line.startswith('  #'):
                # The hashtag marks the header, so we know that we should start reading data from the 
                # following line
                bool_data = True
                continue
            else:
                continue
        
        else:
            # Start looking at the data from here

            # Split by one or more instances of whitespace
            split_line = re.split(pattern = '\s+', string = line)

            # Columns 2 and 5 (0-based) contain the position and the secondary structure annotation
            # However, positions with no annotation in column 5 will skip that column, so we need to
            # make sure the value in that column is a secondary structure annotation

            # If column 2 is "!*", it means we have finished with the first subunit. I will stop there since 
            # the structure is a symmetric homomer
            if split_line[2] == '!*' or split_line[2] == '!':
                break
            elif split_line[2] == '!':
                continue
            else:
                position = int(split_line[2])
                sec_struc = split_line[5]
            
            # Save solvent accessibility
            solv_acc = int(line[34:39].strip())

            # If this is a secondary structure annotation
            if sec_struc in ('H', 'B', 'E', 'G', 'I', 'T', 'S'):
                sec_struc_annotations[position] = [position, code_dict[sec_struc], solv_acc]
            else:
                sec_struc_annotations[position] = [position, 'none', solv_acc]

    
    # Finished looping through the file
    handle.close()
    
    # Save a table with the data
    df_sec_struc = pd.DataFrame.from_dict(sec_struc_annotations, orient='index', columns = ['Position', 'Secondary_structure', 'Solvent_accessibility'])
    df_sec_struc.to_csv(path_or_buf = os.path.join(out_folder, pdb_id + '_dssp_table.txt'), sep = '\t', index = False)
    
    ## Prepare the dictionary with the summary for each structure ##
    summary_dict[pdb_id] = {'Structure' : pdb_id, 
        'none' : 0, 'Missing' : 0, 'Alpha helix' : 0,
        'Beta bridge' : 0, 'Beta ladder' : 0, '3/10 helix' : 0, '5-helix' : 0, 'H-bonded turn' : 0,
        'Bend' : 0
    }
        
    ## Load the PDB file with the indicated regions
    pdb_interface_file = os.path.join(path_interfaces, pdb_id, 'dist_regions_' + pdb_id + '_bio_check_Repair.pdb')
    
    if os.path.exists(pdb_interface_file):
    
        dict_regions = {}

        ## Make a dictionary for the region of each position
        ## Loop through the lines
        with open(pdb_interface_file, 'r') as file_handle:
            # Parse pdb lines
            for line in file_handle:
                split_line = parse_pdb_line(line)
                # atom, atom_num, atom_name, resname, chain, res_num, x, y, z, bfactor

                res_num = int(split_line[5]) 
                region = float(split_line[9])

                # Save residue position and region
                dict_regions[res_num] = region
    
    summary_dict_interfaces[pdb_id] = {'Structure' : pdb_id, 
        'none' : 0, 'Missing' : 0, 'Alpha helix' : 0,
        'Beta bridge' : 0, 'Beta ladder' : 0, '3/10 helix' : 0, '5-helix' : 0, 'H-bonded turn' : 0,
        'Bend' : 0
    }
    
    # Loop through the residues and add to the count of each type of secondary structure
    for key, value_list in sec_struc_annotations.items():
        position = int(key)
        sec_struc_new = value_list[1]
        summary_dict[pdb_id][sec_struc_new] = summary_dict[pdb_id][sec_struc_new] + 1
    
        ## Use a second dictionary only for interface residues
        if os.path.exists(pdb_interface_file):

            if dict_regions.get(position, 0) in [0.75, 1.00]:
                summary_dict_interfaces[pdb_id][sec_struc_new] = summary_dict_interfaces[pdb_id][sec_struc_new] + 1
    


104
104


In [23]:
# Convert the summary dictionary to a data frame
# Save a table with the data
df_summary = pd.DataFrame.from_dict(summary_dict, orient='index')

In [24]:
df_summary

,Structure,none,Missing,Alpha helix,Beta bridge,Beta ladder,3/10 helix,5-helix,H-bonded turn,Bend
4o48,4o48,34,0,68,1,22,4,0,14,13
1b57,1b57,33,0,88,0,28,6,0,16,11
1qq2,1qq2,39,0,40,4,39,7,0,23,21
1dth,1dth,42,0,67,2,39,7,0,26,19
1m7p,1m7p,54,0,93,2,36,13,0,31,17
...,...,...,...,...,...,...,...,...,...,...
1c1f,1c1f,29,0,0,2,77,3,0,14,10
2dfj,2dfj,56,0,79,1,46,15,0,44,26
1a8l,1a8l,37,0,87,1,51,6,0,26,18
1edt,1edt,50,0,76,1,68,15,0,33,22


In [25]:

# df_summary
df_summary.to_csv(path_or_buf = os.path.join(out_folder, 'summary_table.txt'), sep = '\t', index = False)


In [26]:
# Convert the summary dictionary to a data frame
# Save a table with the data
df_summary_interfaces = pd.DataFrame.from_dict(summary_dict_interfaces, orient='index')
df_summary_interfaces.to_csv(path_or_buf = os.path.join(out_folder, 'summary_table_interfaces.txt'), sep = '\t', index = False)